In [2]:
#%%
import torch
import models
# import wrapers
from torch.utils.data import DataLoader
# import load_data
import datetime
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.plugins.environments import LightningEnvironment
import json
import os

In [3]:
def train():
    ### JSON File contains full information about entire run (model, data, hyperparameters)
    ### TODO 
    MODEL_NAME = "CONVERGENCE_AUTO_ENCODER_1"
    config = json.load(open('confmod_auto_encoder.json'))[MODEL_NAME]
    # MODEL_NAME = "AUTO_ENCODER_1"
    # config = json.load(open('confmod_auto_encoder.json'))[MODEL_NAME]

    ''' Dataloading '''
    ### > Separate training and validation HDF5 files 
    rand_samples = torch.randint(0,1000,(100,), dtype=int)
    sample_idxs = torch.ones(100, dtype=int) * 941
    sample_idx = torch.cat([sample_idxs, rand_samples], axis=0)
    sample_idx, _ = torch.sort(sample_idx)

    ld = __import__("load_data")
    train_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "valid", target_sample = sample_idx)
    train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
#     validation_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "valid")
#     validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)

    ### > Single HDF5 file containing training and validation data 
    # data_set = load_data.Dataset_ae(config)
    # # train_set, validation_set, unused_set = torch.utils.data.random_split(data_set, [int(data_set.__len__()*0.3), int(data_set.__len__()*0.05), int(data_set.__len__()*0.65)], generator=torch.Generator().manual_seed(42))
    # train_set, validation_set = torch.utils.data.random_split(data_set, [int(data_set.__len__()*0.8), int(data_set.__len__()*0.2)], generator=torch.Generator().manual_seed(42))
    # train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    # validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)


    ''' Model setup '''
    wrapers = __import__("wrapers")
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)


    ''' Model loading from save file '''
    if config["continue"] == True:
        SAVEPATH = config["SAVEPATH"]
        checkpoint = torch.load(SAVEPATH)
        model.load_state_dict(checkpoint['state_dict'])
        print(" >>> Loaded checkpoint")


    ''' Logging and saving '''
    DATA_NAME = os.path.splitext(os.path.basename(config["PATH_TRAIN"]))[0]

    PATH = ""
    CONFIGURATION = f"../saves/{DATA_NAME}/save_{config['MODEL_NAME']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    # CONFIGURATION = f"../saves/save_{config['MODEL_NAME']}_Nodes{config['n_nodes']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    logger = TensorBoardLogger(PATH, name=CONFIGURATION)


    # '''Define (pytorch_lightning) Trainer '''
    # ### > SLURM Training
#     trainer = pl.Trainer(max_epochs=config["epochs"], accelerator=config["device_type"], devices=config["devices"], num_nodes=config["num_nodes"], strategy='ddp', logger=logger)
    # ### > Jupyter Notebook Training
    # trainer = pl.Trainer(max_epochs=5000, accelerator='gpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    # ### > Jupyter Notebook CPU Training
    trainer = pl.Trainer(max_epochs=2000, accelerator='cpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    
    ''' Train '''
#     trainer.fit(model, train_dataloader, validation_dataloader)
    trainer.fit(model, train_dataloader)

    ''' Saving configuration file into log folder ''' 
    LOGDIR = trainer.log_dir
    json_object = json.dumps(config, indent=4)
    with open(LOGDIR+"/config.json", "w") as outfile:
        outfile.write(json_object)




def main():
    train()

if __name__ == '__main__':
    main()

/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/code/load_data.py:166: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  data_in = torch.tensor(data_in)#[None] # BATCH DIMENSION
/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/code/load_data.py:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data_target = torch.cat([torch.tensor(data_in.real, dtype=torch.float32), torch.tensor(data_in.imag, dtype=torch.float32)], axis=1)
/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/code/load_data.py:172: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5000` reached.


In [4]:
import numpy as np
x = np.zeros((3,3,3))
print(x.shape)
print(np.random.rand(*x.shape))

(3, 3, 3)
[[[0.15095104 0.91295823 0.94432776]
  [0.54219063 0.69414251 0.18925607]
  [0.88253536 0.75250278 0.20890615]]

 [[0.08338908 0.89899906 0.79755765]
  [0.76296112 0.21669249 0.68776967]
  [0.76097889 0.10513168 0.18541401]]

 [[0.34356213 0.37234785 0.18116331]
  [0.45682426 0.16098292 0.68639817]
  [0.78312005 0.00743915 0.81195064]]]
